# Линейная регрессия и стохастический градиентный спуск

Задание основано на материалах лекций по линейной регрессии и градиентному спуску. Вы будете прогнозировать выручку компании в зависимости от уровня ее инвестиций в рекламу по TV, в газетах и по радио.

## Вы научитесь:
- решать задачу восстановления линейной регрессии
- реализовывать стохастический градиентный спуск для ее настройки
- решать задачу линейной регрессии аналитически

## Введение
Линейная регрессия - один из наиболее хорошо изученных методов машинного обучения, позволяющий прогнозировать значения количественного признака в виде линейной комбинации прочих признаков с параметрами - весами модели. Оптимальные (в смысле минимальности некоторого функционала ошибки) параметры линейной регрессии можно найти аналитически с помощью нормального уравнения или численно с помощью методов оптимизации.  

Линейная регрессия использует простой функционал качества - среднеквадратичную ошибку. Мы будем работать с выборкой, содержащей 3 признака. Для настройки параметров (весов) модели решается следующая задача:
$$\Large \frac{1}{\ell}\sum_{i=1}^\ell{{((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}^2} \rightarrow \min_{w_0, w_1, w_2, w_3},$$
где $x_{i1}, x_{i2}, x_{i3}$ - значения признаков $i$-го объекта, $y_i$ - значение целевого признака $i$-го объекта, $\ell$ - число объектов в обучающей выборке.

## Градиентный спуск
Параметры $w_0, w_1, w_2, w_3$, по которым минимизируется среднеквадратичная ошибка, можно находить численно с помощью градиентного спуска.
Градиентный шаг для весов будет выглядеть следующим образом:
$$\Large w_0 \leftarrow w_0 - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$
$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}},\ j \in \{1,2,3\}$$
Здесь $\eta$ - параметр, шаг градиентного спуска.

## Стохастический градиентный спуск
Проблема градиентного спуска, описанного выше, в том, что на больших выборках считать на каждом шаге градиент по всем имеющимся данным может быть очень вычислительно сложно. 
В стохастическом варианте градиентного спуска поправки для весов вычисляются только с учетом одного случайно взятого объекта обучающей выборки:
$$\Large w_0 \leftarrow w_0 - \frac{2\eta}{\ell} {((w_0 + w_1x_{k1} + w_2x_{k2} +  w_3x_{k3}) - y_k)}$$
$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} {x_{kj}((w_0 + w_1x_{k1} + w_2x_{k2} +  w_3x_{k3}) - y_k)},\ j \in \{1,2,3\},$$
где $k$ - случайный индекс, $k \in \{1, \ldots, \ell\}$.

## Нормальное уравнение 
Нахождение вектора оптимальных весов $w$ может быть сделано и аналитически.
Мы хотим найти такой вектор весов $w$, чтобы вектор $y$, приближающий целевой признак, получался умножением матрицы $X$ (состоящей из всех признаков объектов обучающей выборки, кроме целевого) на вектор весов $w$. То есть, чтобы выполнялось матричное уравнение:
$$\Large y = Xw$$
Домножением слева на $X^T$ получаем:
$$\Large X^Ty = X^TXw$$
Это хорошо, поскольку теперь матрица $X^TX$ - квадратная, и можно найти решение (вектор $w$) в виде:
$$\Large w = {(X^TX)}^{-1}X^Ty$$
Матрица ${(X^TX)}^{-1}X^T$ - [*псевдообратная*](https://ru.wikipedia.org/wiki/Псевдообратная_матрица) для матрицы $X$. В NumPy такую матрицу можно вычислить с помощью функции [numpy.linalg.pinv](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.linalg.pinv.html).

Однако, нахождение псевдообратной матрицы - операция вычислительно сложная и нестабильная в случае малого определителя матрицы $X$ (проблема мультиколлинеарности). 
На практике лучше находить вектор весов $w$ решением матричного уравнения 
$$\Large X^TXw = X^Ty$$Это может быть сделано с помощью функции [numpy.linalg.solve](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linalg.solve.html).

Но все же на практике для больших матриц $X$ быстрее работает градиентный спуск, особенно его стохастическая версия.

## Инструкции по выполнению

В начале напишем простую функцию для записи ответов в текстовый файл. Ответами будут числа, полученные в ходе решения этого задания, округленные до 3 знаков после запятой. Полученные файлы после выполнения задания надо отправить в форму на странице задания на Coursera.org.

In [86]:
def write_answer_to_file(answer, filename):
    with open(filename, 'w') as f_out:
        f_out.write(str(round(answer, 3)))

**1. Загрузите данные из файла *advertising.csv* в объект pandas DataFrame. [Источник данных](http://www-bcf.usc.edu/~gareth/ISL/data.html).**

In [87]:
import pandas as pd
adver_data = pd.read_csv('advertising.csv')

**Посмотрите на первые 5 записей и на статистику признаков в этом наборе данных.**

In [88]:
# Ваш код здесь
adver_data.head()

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [89]:
# Ваш код здесь
adver_data.describe()

,TV,Radio,Newspaper,Sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,14.022500
std,85.854236,14.846809,21.778621,5.217457
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,10.375000
50%,149.750000,22.900000,25.750000,12.900000
75%,218.825000,36.525000,45.100000,17.400000
max,296.400000,49.600000,114.000000,27.000000


**Создайте массивы NumPy *X* из столбцов TV, Radio и Newspaper и *y* - из столбца Sales. Используйте атрибут *values* объекта pandas DataFrame.**

In [90]:
# Ваш код здесь
import numpy as np
X = adver_data.values[:,:3]
Y = adver_data.values[:,-1:]
print(X[:5])
print(Y[:5])

[[ 230.1   37.8   69.2]
 [  44.5   39.3   45.1]
 [  17.2   45.9   69.3]
 [ 151.5   41.3   58.5]
 [ 180.8   10.8   58.4]]
[[ 22.1]
 [ 10.4]
 [  9.3]
 [ 18.5]
 [ 12.9]]


In [91]:
X[0,1]

37.799999999999997

**Отмасштабируйте столбцы матрицы *X*, вычтя из каждого значения среднее по соответствующему столбцу и поделив результат на стандартное отклонение. Для определенности, используйте методы mean и std векторов NumPy (реализация std в Pandas может отличаться). Обратите внимание, что в numpy вызов функции .mean() без параметров возвращает среднее по всем элементам массива, а не по столбцам, как в pandas. Чтобы произвести вычисление по столбцам, необходимо указать параметр axis.**

In [92]:
means = np.mean(X, axis=0)
print (means)
stds = np.std(X, axis=0)
print (stds)

[ 147.0425   23.264    30.554 ]
[ 85.63933176  14.80964564  21.72410606]


In [93]:
#X = # Ваш код здесь
print (X.shape)
for i in range(0,X.shape[0]-1):
    for j in range(0,X.shape[1]-1):
        X[i,j] -= means[j]
        X[i,j] /= stds[j]
print(X[:5])

(200, 3)
[[  9.69852266e-01   9.81522472e-01   6.92000000e+01]
 [ -1.19737623e+00   1.08280781e+00   4.51000000e+01]
 [ -1.51615499e+00   1.52846331e+00   6.93000000e+01]
 [  5.20496822e-02   1.21785493e+00   5.85000000e+01]
 [  3.94182198e-01  -8.41613655e-01   5.84000000e+01]]


**Добавьте к матрице *X* столбец из единиц, используя методы *hstack*, *ones* и *reshape* библиотеки NumPy. Вектор из единиц нужен для того, чтобы не обрабатывать отдельно коэффициент $w_0$ линейной регрессии.**

In [94]:
import numpy as np
ones_column = np.ones(X.shape[0]).reshape([X.shape[0],1])
X = np.hstack((X, ones_column)) # Ваш код здесь

In [95]:
print(X) 

[[  9.69852266e-01   9.81522472e-01   6.92000000e+01   1.00000000e+00]
 [ -1.19737623e+00   1.08280781e+00   4.51000000e+01   1.00000000e+00]
 [ -1.51615499e+00   1.52846331e+00   6.93000000e+01   1.00000000e+00]
 [  5.20496822e-02   1.21785493e+00   5.85000000e+01   1.00000000e+00]
 [  3.94182198e-01  -8.41613655e-01   5.84000000e+01   1.00000000e+00]
 [ -1.61540845e+00   1.73103399e+00   7.50000000e+01   1.00000000e+00]
 [ -1.04557682e+00   6.43904671e-01   2.35000000e+01   1.00000000e+00]
 [ -3.13436589e-01  -2.47406325e-01   1.16000000e+01   1.00000000e+00]
 [ -1.61657614e+00  -1.42906863e+00   1.00000000e+00   1.00000000e+00]
 [  6.16042873e-01  -1.39530685e+00   2.12000000e+01   1.00000000e+00]
 [ -9.45155670e-01  -1.17923146e+00   2.42000000e+01   1.00000000e+00]
 [  7.90028350e-01   4.96973404e-02   4.00000000e+00   1.00000000e+00]
 [ -1.43908760e+00   7.99208859e-01   6.59000000e+01   1.00000000e+00]
 [ -5.78501712e-01  -1.05768905e+00   7.20000000e+00   1.00000000e+00]
 [  6.

**2. Реализуйте функцию *mserror* - среднеквадратичную ошибку прогноза. Она принимает два аргумента - объекты Series *y* (значения целевого признака) и *y\_pred* (предсказанные значения). Не используйте в этой функции циклы - тогда она будет вычислительно неэффективной.**

In [96]:
def mserror(y, y_pred):
    # Ваш код здесь
    return 1/len(y)*sum((y-y_pred)**2)

In [101]:
#test
y = np.array([1,2,3])
y_pred = 4
print(mserror(y, y_pred))

4.66666666667


**Какова среднеквадратичная ошибка прогноза значений Sales, если всегда предсказывать медианное значение Sales по исходной выборке? Запишите ответ в файл '1.txt'.**

In [105]:
answer1 = mserror(np.median(Y,axis=0), Y)# Ваш код здесь
print(np.median(Y,axis=0))
print(answer1)
write_answer_to_file(answer1[0], '1.txt')

[ 12.9]
[ 5669.15]


**3. Реализуйте функцию *normal_equation*, которая по заданным матрицам (массивам NumPy) *X* и *y* вычисляет вектор весов $w$ согласно нормальному уравнению линейной регрессии.**

$$\Large w = {(X^TX)}^{-1}X^Ty$$

In [109]:
def normal_equation(X, y):
    return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y)  # Ваш код здесь

In [110]:
norm_eq_weights = normal_equation(X, y)
print(norm_eq_weights)

[[ -8.99223828e-02]
 [  2.78467664e+00]
 [  1.09088228e-02]
 [  1.36595726e+01]]


**Какие продажи предсказываются линейной моделью с весами, найденными с помощью нормального уравнения, в случае средних инвестиций в рекламу по ТВ, радио и в газетах? (то есть при нулевых значениях масштабированных признаков TV, Radio и Newspaper). Запишите ответ в файл '2.txt'.**

In [132]:
one = np.ones(1)
means_one = np.hstack((means, one)) # Ваш код здесь
print(means_one)
answer2 = means_one.dot(norm_eq_weights)
print(answer2)
write_answer_to_file(answer2[0], '2.txt')

[ 147.0425   23.264    30.554     1.    ]
[ 65.55318619]


**4. Напишите функцию *linear_prediction*, которая принимает на вход матрицу *X* и вектор весов линейной модели *w*, а возвращает вектор прогнозов в виде линейной комбинации столбцов матрицы *X* с весами *w*.**
$$\Large \frac{1}{\ell}\sum_{i=1}^\ell{{((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}^2} \rightarrow \min_{w_0, w_1, w_2, w_3},$$

In [123]:
def linear_prediction(X, w):
    return X.dot(w)

**Какова среднеквадратичная ошибка прогноза значений Sales в виде линейной модели с весами, найденными с помощью нормального уравнения? Запишите ответ в файл '3.txt'.**

In [133]:
answer3 = mserror(Y, linear_prediction(X,norm_eq_weights))# Ваш код здесь
print(answer3)
write_answer_to_file(answer3[0], '3.txt')

[ 18.83774002]


**5. Напишите функцию *stochastic_gradient_step*, реализующую шаг стохастического градиентного спуска для линейной регрессии. Функция должна принимать матрицу *X*, вектора *y* и *w*, число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов, а также число *$\eta$* (eta) - шаг градиентного спуска (по умолчанию *eta*=0.01). Результатом будет вектор обновленных весов. Наша реализация функции будет явно написана для данных с 3 признаками, но несложно модифицировать для любого числа признаков, можете это сделать.**

## Стохастический градиентный спуск
Проблема градиентного спуска, описанного выше, в том, что на больших выборках считать на каждом шаге градиент по всем имеющимся данным может быть очень вычислительно сложно. 
В стохастическом варианте градиентного спуска поправки для весов вычисляются только с учетом одного случайно взятого объекта обучающей выборки:
$$\Large w_0 \leftarrow w_0 - \frac{2\eta}{\ell} {((w_0 + w_1x_{k1} + w_2x_{k2} +  w_3x_{k3}) - y_k)}$$
$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} {x_{kj}((w_0 + w_1x_{k1} + w_2x_{k2} +  w_3x_{k3}) - y_k)},\ j \in \{1,2,3\},$$
где $k$ - случайный индекс, $k \in \{1, \ldots, \ell\}$.

In [216]:
def stochastic_gradient_step(X, y, w, train_ind, eta=0.01):
    grad0 = 2*eta*((w[0,0]*X[train_ind,0]+w[0,1]*X[train_ind,1]
                    +w[0,2]*X[train_ind,2]+w[0,3]*X[train_ind,3])-y[train_ind])
    grad1 = grad0*X[train_ind,0]
    grad2 = grad0*X[train_ind,1]
    grad3 = grad0*X[train_ind,2]
    #print(w)
    #print(np.reshape(np.array([grad0, grad1, grad2, grad3]),(1,4)))
   # print(w-eta*np.reshape(np.array([grad0, grad1, grad2, grad3]),(1,4)))
    return  w-eta*np.reshape(np.array([grad0, grad1, grad2, grad3]),(1,4))

**6. Напишите функцию *stochastic_gradient_descent*, реализующую стохастический градиентный спуск для линейной регрессии. Функция принимает на вход следующие аргументы:**
- X - матрица, соответствующая обучающей выборке
- y - вектор значений целевого признака
- w_init - вектор начальных весов модели
- eta - шаг градиентного спуска (по умолчанию 0.01)
- max_iter - максимальное число итераций градиентного спуска (по умолчанию 10000)
- max_weight_dist - максимальное евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,
при котором алгоритм прекращает работу (по умолчанию 1e-8)
- seed - число, используемое для воспроизводимости сгенерированных псевдослучайных чисел (по умолчанию 42)
- verbose - флаг печати информации (например, для отладки, по умолчанию False)

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

In [236]:
import scipy
from scipy import spatial
def stochastic_gradient_descent(X, y, w_init, eta=1e-2, max_iter=1e4,
                                min_weight_dist=1e-8, seed=42, verbose=False):
    # Инициализируем расстояние между векторами весов на соседних
    # итерациях большим числом. 
    max_iter = 1
    weight_dist = np.inf
    # Инициализируем вектор весов
    w = w_init.reshape((1,4))
    # Сюда будем записывать ошибки на каждой итерации
    errors = []
    # Счетчик итераций
    iter_num = 0
    # Будем порождать псевдослучайные числа 
    # (номер объекта, который будет менять веса), а для воспроизводимости
    # этой последовательности псевдослучайных чисел используем seed.
    np.random.seed(seed)
        
    # Основной цикл
    while weight_dist > min_weight_dist and iter_num < max_iter:
        # порождаем псевдослучайный 
        # индекс объекта обучающей выборки
        random_ind = np.random.randint(X.shape[0])
        
        w_new = stochastic_gradient_step(X, y, w, random_ind, 0.01)
        weight_dist = scipy.spatial.distance.cosine(w, w_new)
        #errors = np.append(errors, mserror(Y, linear_prediction(X,w_new)))
        iter_num += 1
        if verbose == True:
            #print(w)
            print(w_new)
            print(weight_dist)
            #print(Y)
            #print(np.reshape(linear_prediction(X,w_new[0]),(200,1)))
            print(mserror(Y, np.reshape(linear_prediction(X,w_new[0]),(200,1)))
        w = w_new
        # Ваш код здесь
        
    return w, errors

SyntaxError: invalid syntax (<ipython-input-236-1f4bab0d1997>, line 37)

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов *w_init*, состоящий из нулей. Оставьте параметры  *eta* и *seed* равными их значениям по умолчанию (*eta*=0.01, *seed*=42 - это важно для проверки ответов).**

In [235]:
%%time
w_init = np.array([1.,1.,1.,1])
stoch_grad_desc_weights, stoch_errors_by_iter =  stochastic_gradient_descent(X, Y, 
                w_init, eta=0.01, max_iter=1000, min_weight_dist=1e-8, seed=42, verbose=True)

[[ 0.9983468   0.9974295   1.00146949  0.96462159]]
0.000114606355701
[[ 22.1]
 [ 10.4]
 [  9.3]
 [ 18.5]
 [ 12.9]
 [  7.2]
 [ 11.8]
 [ 13.2]
 [  4.8]
 [ 10.6]
 [  8.6]
 [ 17.4]
 [  9.2]
 [  9.7]
 [ 19. ]
 [ 22.4]
 [ 12.5]
 [ 24.4]
 [ 11.3]
 [ 14.6]
 [ 18. ]
 [ 12.5]
 [  5.6]
 [ 15.5]
 [  9.7]
 [ 12. ]
 [ 15. ]
 [ 15.9]
 [ 18.9]
 [ 10.5]
 [ 21.4]
 [ 11.9]
 [  9.6]
 [ 17.4]
 [  9.5]
 [ 12.8]
 [ 25.4]
 [ 14.7]
 [ 10.1]
 [ 21.5]
 [ 16.6]
 [ 17.1]
 [ 20.7]
 [ 12.9]
 [  8.5]
 [ 14.9]
 [ 10.6]
 [ 23.2]
 [ 14.8]
 [  9.7]
 [ 11.4]
 [ 10.7]
 [ 22.6]
 [ 21.2]
 [ 20.2]
 [ 23.7]
 [  5.5]
 [ 13.2]
 [ 23.8]
 [ 18.4]
 [  8.1]
 [ 24.2]
 [ 15.7]
 [ 14. ]
 [ 18. ]
 [  9.3]
 [  9.5]
 [ 13.4]
 [ 18.9]
 [ 22.3]
 [ 18.3]
 [ 12.4]
 [  8.8]
 [ 11. ]
 [ 17. ]
 [  8.7]
 [  6.9]
 [ 14.2]
 [  5.3]
 [ 11. ]
 [ 11.8]
 [ 12.3]
 [ 11.3]
 [ 13.6]
 [ 21.7]
 [ 15.2]
 [ 12. ]
 [ 16. ]
 [ 12.9]
 [ 16.7]
 [ 11.2]
 [  7.3]
 [ 19.4]
 [ 22.2]
 [ 11.5]
 [ 16.9]
 [ 11.7]
 [ 15.5]
 [ 25.4]
 [ 17.2]
 [ 11.7]
 [ 23.8]
 [ 14.8]
 [ 

**Посмотрим, чему равна ошибка на первых 50 итерациях стохастического градиентного спуска. Видим, что ошибка не обязательно уменьшается на каждой итерации.**

In [ ]:
%pylab inline
plot(range(50), stoch_errors_by_iter[:50])
xlabel('Iteration number')
ylabel('MSE')

**Теперь посмотрим на зависимость ошибки от номера итерации для $10^5$ итераций стохастического градиентного спуска. Видим, что алгоритм сходится.**

In [ ]:
%pylab inline
plot(range(len(stoch_errors_by_iter)), stoch_errors_by_iter)
xlabel('Iteration number')
ylabel('MSE')

**Посмотрим на вектор весов, к которому сошелся метод.**

In [ ]:
stoch_grad_desc_weights

**Посмотрим на среднеквадратичную ошибку на последней итерации.**

In [ ]:
stoch_errors_by_iter[-1]

**Какова среднеквадратичная ошибка прогноза значений Sales в виде линейной модели с весами, найденными с помощью градиентного спуска? Запишите ответ в файл '4.txt'.**

In [ ]:
answer4 = # Ваш код здесь
print(answer4)
write_answer_to_file(answer4, '4.txt')

**Ответами к заданию будут текстовые файлы, полученные в ходе этого решения. Обратите внимание, что отправленные файлы не должны содержать пустую строку в конце. Данный нюанс является ограничением платформы Coursera. Мы работаем над исправлением этого ограничения.**